In [ ]:
import random
import matplotlib.image as mpimg
from nltk import pos_tag
from wordcloud import WordCloud
import pandas as pd
import os
import time
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, add, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from nltk.translate.bleu_score import corpus_bleu
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import pickle
import cv2
import random
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from nltk.tokenize import word_tokenize

from datasetLoader import datasetLoader

import Model from Model


In [ ]:
df = pd.read_table("data_results.csv")
df.head()

In [ ]:
# Rename columns with a new mapping
columns_mapping = {
    " comment_number" : "comment_number",
    " comment" : "comment"
}
dataframe = df.rename(columns=columns_mapping)

# Group data by image name and aggregate comments into lists
grouped = dataframe.groupby('image_name')['comment'].apply(lambda x: x.tolist()).reset_index(name='comments')

# Generate image paths
image_file_paths = ['/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images/' + img_name for img_name in grouped['image_name']]
captions = grouped['comments']

# Calculate unique counts and total records
num_unique_images = dataframe['image_name'].nunique()
num_unique_comments = dataframe['comment'].nunique()
num_total_records = len(dataframe)

# Print the results
print(f"Number of unique images: {num_unique_images}")
print(f"Number of unique comments: {num_unique_comments}")
print(f"Total number of records: {num_total_records}")


In [ ]:
from keras.preprocessing.text import Tokenizer

def preprocess_text(dataset):
    # Clean the captions in the dataset
    dataset['description'] = dataset['description'].apply(lambda text: text.lower())
    dataset['description'] = dataset['description'].apply(lambda text: text.replace("[^A-Za-z]", ""))
    dataset['description'] = dataset['description'].apply(lambda text: text.replace("\s+", " "))
    dataset['description'] = dataset['description'].apply(lambda text: " ".join([word for word in text.split() if len(word) > 1]))
    dataset['description'] = "startseq " + dataset['description'] + " endseq"  # Add startseq and endseq
    return dataset


processed_data = preprocess_text(dataframe)
descriptions_list = processed_data['description'].tolist()


text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(descriptions_list)
vocabulary_size = len(text_tokenizer.word_index) + 1
print(vocabulary_size)

max_caption_length = max(len(description.split()) for description in descriptions_list)

image_files = processed_data['image'].unique().tolist()
total_images = len(image_files)


split_idx = round(0.7 * total_images)
train_image_files = image_files[:split_idx]
test_image_files = image_files[split_idx:]

train_set = processed_data[processed_data['image'].isin(train_image_files)]
test_set = processed_data[processed_data['image'].isin(test_image_files)]


train_set.reset_index(inplace=True, drop=True)
test_set.reset_index(inplace=True, drop=True)




In [ ]:
train_generator, test_generator, validation_generator = datasetLoader(train_set, test_set, 50, 3)
caption_model = Model("clip")

optimizer = Adam(learning_rate=0.001)
caption_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = caption_model.fit(
        train_generator,
        epochs=50,
        validation_data=validation_generator)


In [ ]:

caption_model.save_weights('model.h5')